In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

from skimage.io import imread
from skimage.transform import resize
from skimage.util import img_as_int

from keras.preprocessing.image import load_img

Crop objectes i afegir-los random
Per balancejar necessitarem + de 1 objecte per imatge
Les bound boxes tenim els extrems top-left i bottom-right (però no ocupen tota la bb)
Utilitzar IoU per mesurar overlapping dels objectes
SMPLR dataaugmentation
https://github.com/meysam-madadi/SMPLR/blob/master/Data/dataaugmentation.py



In [1]:
from dataset import Dataset
import os

root_path = './'
train_val_path = os.path.join(root_path, 'VOCtrainval_06-Nov-2007', 'VOCdevkit', 'VOC2007')
test_path = os.path.join('VOCtest_06-Nov-2007', 'VOCdevkit', 'VOC2007')

dataset = Dataset(train_val_path, test_path, 224)

In [2]:
train_data, val_data, segmentation_data = dataset.generate_train_val_split()

print(f'Train length: {len(train_data)}')
print(f'Validation length: {len(val_data)}')
print(f'Segmentation length: {len(segmentation_data)}')

Train length: 4110
Validation length: 901
Segmentation length: 422


In [ ]:
test_data = dataset.get_test_data()

print(f'Test length: {len(test_data)}')

In [ ]:
import cv2
import numpy as np
from skimage.io import imread
from skimage.color import rgb2gray
from skimage import img_as_ubyte, img_as_float

In [ ]:
path = './VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/SegmentationObject/000032.png'

seg_image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
print(np.unique(seg_image))

In [ ]:
plt.imshow(seg_image, cmap='gray')

In [ ]:
def _generate_mask_from_segmentation(segmentation_image):
    unique_values = np.unique(segmentation_image)
    sorted_values = np.sort(unique_values)
    print(sorted_values)
    sorted_values = sorted_values[1:]
    sorted_values[::-1].sort()
    print(sorted_values)
    sorted_values = sorted_values[1:]
    print(sorted_values)
    selected_value = np.random.choice(sorted_values)
    print(selected_value)

    segmentation_mask = (segmentation_image == selected_value).astype(np.float32)
    segmentation_mask = cv2.resize(segmentation_mask, (224, 224))
    return segmentation_mask


In [ ]:
mask = _generate_mask_from_segmentation(seg_image)

In [ ]:
print(np.max(seg_image))
print(np.min(seg_image))

print(np.max(mask))
print(np.min(mask))

In [ ]:
plt.imshow(mask, cmap='gray')

In [ ]:
np.any(mask == 1.0)
indices = np.where(mask == 1.0)
print(min(indices[0]))
print(min(indices[1]))
print(max(indices[0]))
print(max(indices[1]))

In [ ]:
image_data = train_data[0]
image = image_data['image']
name = image_data['name']
boxes = image_data['boxes']

object_image = train_data[13]['image']

In [ ]:
path_2 = './VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages/000005.jpg'

image = cv2.imread(path_2)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = img_as_float(cv2.resize(image, (224, 224)))

In [ ]:
plt.imshow(image)

In [ ]:
path_3 = './VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages/000032.jpg'

object_image = cv2.imread(path_3)
object_image = cv2.cvtColor(object_image, cv2.COLOR_BGR2RGB)
object_image = img_as_float(cv2.resize(object_image, (224, 224)))

print(object_image.shape)
print(image.shape)

In [ ]:
plt.imshow(object_image)

In [ ]:
def _add_object_to_image(image, object_image, segmentation_image):
    mask = _generate_mask_from_segmentation(segmentation_image)
    mask = np.repeat(np.expand_dims(mask, axis=-1), 3, axis=-1)
    image = image*(1 - mask) + object_image*mask
    return image

In [ ]:
mask_2 = np.repeat(np.expand_dims(mask, axis=-1), 3, axis=-1)
print(mask_2.shape)

In [ ]:
image_distort = _add_object_to_image(image, object_image, seg_image)

plt.imshow(image_distort)

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

def _get_segmentation(dataset_path, image_name):
    path = os.path.join(dataset_path, 'SegmentationObject', f'{image_name}.png')
    segmentation = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    return segmentation


def _create_segmentation_masks(segmentation_image):
    unique_values = np.unique(segmentation_image)
    sorted_values = np.sort(unique_values)
    # Remove pixels with value 0 (i.e. background)
    sorted_values = sorted_values[1:]
    sorted_values[::-1].sort()
    # Remove pixels with value 220 (i.e. object border)
    sorted_values = sorted_values[1:]
    print(sorted_values)

    masks = []
    boxes = []

    for value in sorted_values:
        mask = (segmentation_image == value).astype(np.float32)
        mask = cv2.resize(mask, (224, 224))
        indices = np.where(mask == 1.0)
        box = [min(indices[0]), min(indices[1]), max(indices[0]), max(indices[1])]
        masks.append(mask)
        boxes.append(box)

    return masks, boxes

In [ ]:
path = './VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007'
image_name = '000032'

segmentation = _get_segmentation(path, image_name)

In [ ]:
masks, boxes = _create_segmentation_masks(segmentation)

In [ ]:
print(boxes)

In [ ]:
plt.imshow(masks[2], cmap='gray')